In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
from os import listdir, mkdir
import pandas as pd

In [5]:
main_dir = '/gdrive/MyDrive/BirdShazam/bird_data/'

In [6]:
import os
os.listdir(main_dir)

['birds.csv', 'to_download.csv', 'wav_files', 'downloaded.csv']

In [10]:
len(os.listdir(main_dir+"/wav_files/train"))

1923

In [7]:
audio_dir = main_dir+'audio_files/'
files = listdir(audio_dir)
len(files)

FileNotFoundError: ignored

In [ ]:
import pandas as pd
import requests
from tqdm import tqdm

country = 'brazil'
response = requests.get(f'https://www.xeno-canto.org/api/2/recordings?query=cnt:{country}')
js = response.json()

df = pd.DataFrame()
for n_page in tqdm(range(1, js["numPages"]+1)):
    response = requests.get(f'https://www.xeno-canto.org/api/2/recordings?query=cnt:{country}&page={n_page}')
    page_json = response.json()
    ids, files, file_names, ens, lengths, gens = [], [], [], [], [], []
    for recording in page_json["recordings"]:
        ids.append(recording["id"])
        files.append(recording["file"])
        file_names.append(recording["file-name"])
        ens.append(recording["en"])
        lengths.append(recording["length"])
        gens.append(recording["gen"])


    df_ = pd.DataFrame.from_records({'id': ids, "file": files, "file-name": file_names, "en": ens, "gen": gens, "length": lengths})
    df = df.append(df_)
    
df.to_csv(dir+"birds.csv", index=False)

  0%|          | 0/118 [00:00<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
from pandas import read_csv
from urllib.request import urlretrieve
from os import listdir, mkdir, path as os_path
from tqdm.notebook import tqdm
from time import sleep

df = read_csv(dir+"birds.csv")

#Removing recordings that are not classified
df = df[df['en'] != 'Identity unknown'].copy()
df = df[df['en'] != 'Soundscape'].copy()

counts = df['en'].value_counts()
chosen = counts.head(25).index
df = df[df["en"].isin(chosen)]
df.to_csv(dir+"to_download.csv", index=False)

print(df.shape)
print(len(df["en"].unique()), "different species")

audio_dir = dir+'audio_files/'
if not os_path.isdir(audio_dir):
    mkdir(audio_dir)

nots = []

for row in tqdm(df.iterrows(), total=df.shape[0]):
    url = row[1]["file"]
    f_name = str(row[1]["id"]) + row[1]["file-name"][-4:].lower()
    try:
        urlretrieve(url, filename=audio_dir + f_name)
    except Exception as e:
        print("\nRetrying:", url)
        print(e) 
        sleep(60)
        try:
            urlretrieve(url, filename=audio_dir + f_name)
        except Exception as ee:
            print("Not downloaded|", f_name)
            print(ee)
            nots.append(row[1]["id"])
            pass

if len(nots) > 0:
    with open(dir+'not_downloaded.txt', 'w') as f:
        for item in nots:
            f.write(str(item) + '\n')
    print(str(nots))
else:
    print('All files were successfully downloaded!')

In [ ]:
!pip install pydub
from sklearn.model_selection import train_test_split
from pydub import AudioSegment
from os import listdir, mkdir, path as os_path, remove as os_remove
from tqdm.notebook import tqdm

#Check downloaded files
files = listdir(audio_dir)
df = pd.read_csv(dir+'to_download.csv')
df['file-name'] = df['file-name'].str.lower()
df['filename'] = df['id'].astype(str) + df['file-name'].str[-4:]
downloaded = df[df['filename'].isin(files)][['filename', 'en']]
downloaded.to_csv(dir+'downloaded.csv', index=False)
print('Not downloaded:', len(df) - len(downloaded))

train, test = train_test_split(downloaded[['filename', 'en']], stratify=downloaded["en"], test_size=0.1)

def to_wav(filename, n_channels, sample_rate, input_dir, output_dir, delmp3 = False):
    input_type = filename[-4:].lower()
    if input_type == '.mp3':
        sound = AudioSegment.from_mp3(f"{input_dir}/{filename}")
        new_fn = fn[:-4] + '.wav'
    elif input_type == '.wav':
        sound = AudioSegment.from_wav(f"{input_dir}/{filename}")
        new_fn = filename
    sound = sound.set_channels(n_channels)
    sound.set_frame_rate(sample_rate)
    sound.export(f"{output_dir}/{new_fn}", format="wav")
    if delmp3:
        if new_fn in listdir(output_dir):
            os_remove(f"{input_dir}/{filename}")
        else:
            print(f'Problema: {input_dir}/{filename}')

wav_dir = dir+'wav_files/'
if not os_path.isdir(wav_dir):
    mkdir(wav_dir)
    mkdir(wav_dir + 'test/')
    mkdir(wav_dir + 'train/')
    
# for fn in tqdm(test['filename']):
#     to_wav(fn, n_channels=1, sample_rate=16000, input_dir=audio_dir, output_dir=wav_dir+'test')

for fn in tqdm(train['filename']):
    try:
        to_wav(fn, n_channels=1, sample_rate=16000, input_dir=audio_dir, output_dir=wav_dir+'train', delmp3=True)
    except Exception as e:
        print(fn)
        print(e)

Not downloaded: 0


  0%|          | 0/4157 [00:00<?, ?it/s]

586414.wav
Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwa

In [ ]:
len(listdir(wav_dir + 'train/'))

1923

In [ ]:
wav_dir ='/gdrive/MyDrive/BirdShazam/bird_data/wav_files/'

# Nova seção

In [ ]:
len(df)

2144

In [ ]:
len(downloaded)

2144

In [ ]:
len(files)

2144

In [ ]:
df[~df['filename'].isin(files)][['filename', 'en']]

,filename,en
18632,593914.mp3,Lesser Woodcreeper
43429,539758.mp3,Blue Manakin


In [ ]:
'678053.mp3' in df['filename'].to_list()

True

In [ ]:
for file in tqdm(files):
    if file not in df['filename'].to_list():
        print(file)

  0%|          | 0/2144 [00:00<?, ?it/s]

593914.wav
539758.wav
